In [189]:
import numpy as np
import pandas as pd
movies=pd.read_csv('tmdb_5000_movies.csv');
credits=pd.read_csv('tmdb_5000_credits.csv');
movies=movies.merge(credits,on='title');
movies.shape
movies.head(1)
#genres,id,keywords,title,overview,cast,crew
movies.info()
movies=movies[['id','title','cast','keywords','overview','crew','genres']]
movies.dropna(inplace=True)
movies.duplicated().sum()
movies.iloc[0].genres
import ast;
def convert(obj):
    list_object=[]
    for i in ast.literal_eval(obj):
        list_object.append(i['name'])
    return list_object;

movies['genres']=movies['genres'].apply(convert);  
movies['keywords']=movies['keywords'].apply(convert); 
movies.head()     
#compute top three cast only
def convert2(obj):
    list_object=[]
    counter=0;
    for i in ast.literal_eval(obj):
        if counter!=3:
           list_object.append(i['name'])
           counter=counter+1
        else:
            break;   
    return list_object;
movies['cast']=movies['cast'].apply(convert2);   

movies.head(1)   

def fetch_director(obj):
    list_object=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
          list_object.append(i['name'])
          break;
    return list_object;

movies['crew']=movies['crew'].apply(fetch_director);
movies.head(1)

movies['overview'][0]
movies['overview']=movies['overview'].apply(lambda x:x.split())
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew'];
new_df=movies[['id','title','tags']];
new_df['tags']=new_df['tags'].apply(lambda x:' '.join(x));
new_df['tags']=new_df['tags'].apply(lambda x:x.lower());
new_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

C:\Users\HP\AppData\Local\Temp\ipykernel_2260\1015197210.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:' '.join(x));
C:\Users\HP\AppData\Local\Temp\ipykernel_2260\1015197210.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower());


,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [190]:
#cv.get_feature_names()
import nltk;
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
#function to create stem data
def stem(text):
    list_obj=[]
    for i in text.split():
        list_obj.append(ps.stem(i))

    return " ".join(list_obj);   

new_df['tags']=new_df['tags'].apply(stem)  

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(new_df['tags']).toarray()

C:\Users\HP\AppData\Local\Temp\ipykernel_2260\913557154.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [193]:
new_df['tags'][0]
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)
similarity[0]
#recommend movies on the basis of given movie
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

recommend('Batman Begins')        

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf
